<div align=center><h1 style="display: inline;" >5. Probabilistic Text Analysis (B1 continued)</h1><img style="display: inline;" src=http://industrypulse.com/wp-content/uploads/2017/05/h2oai-1068x1068.png width=100></div>


Here, we compare the performance of the word2vec regression model with a competing model. This model is based on first calculating $p(y | x_i)$ for each independent word, $x_i$. The predicting distribution for an given job description is then generated from the component words:

<p><div align=center>$
p(y | x_1, \dots, x_n) \varpropto \prod_{i=1}^n p(y \mid x_i)
$</div></p>

The distributions of incomes for component words are assumed to follow lognormal distribitions — that is, the log-incomes are assumed to be normally distributed, with unknown means and variances. Empirically, all but the highest incomes are <a href=https://ideas.repec.org/p/wpa/wuwpmi/0505006.html>observed</a> to be roughly lognormally distributed.

**Correct, Bayesian, method:** the problem is estimating the parameters, $\boldsymbol{\theta}$ of these normal distributions (of log-income). Given that the likelihood function is Gaussian, we can invoke a Bayesian conjugate prior: 

<p><div align=center>$p(\boldsymbol{\theta} | y) \varpropto p(y | \boldsymbol{\theta}) p(\boldsymbol{\theta})$</div></p>

which <a href=https://www.cs.ubc.ca/~murphyk/Papers/bayesGauss.pdf>is</a> a Normal-Gamma distribution over the mean and precision ($\mu$ and $\lambda$):
<p><div align=center>$NG(\mu,\lambda | \mu_0, \kappa_0, \alpha_0, \beta_0) = \mathcal{N}\left(\mu | \mu_0, (\kappa_0 \lambda_0)^{-1}\right) \Gamma(\lambda_0 | \alpha_0, \beta_0)$</div></p>

The technically correct way to proceed would be to calculate these four parameters for each word, resulting in a t-distribution over log-income for each word. These t-distributions would then be multiplied to give a combined distribution (and mean).

**Quick approximation:** Here, for each word, we estimate the standard error of the mean (log-income) as:
<p><div align=center>$SE \approx \frac{s}{\sqrt{n}}$</div></p>
where $s$ is the standard deviation of the sample, and $n$ is the sample size.

We start by importing the data:

In [3]:
import pandas as pd
import string
lending = pd.read_csv("../data/LoanStats_2017Q1.csv")
lending = lending[['emp_title','annual_inc']]
lending = lending.dropna()
lending = lending[lending['annual_inc'] < 200000]
lending['emp_title'] = lending['emp_title'].str.lower()
#for i in lending.index:
#   lending['emp_title'][i] = lending['emp_title'][i].translate(None, string.punctuation)

Just for curiosity, I sampled the number of words in job titles:

In [7]:
jobs['nwords'] = 0
for i in jobs.index[:1000]:
    jobs['nwords'][i] = len(jobs['words'][i])
jobs.head(20)
jobs[:1000].groupby('nwords').count()

/usr/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,emp_title,annual_inc,words
nwords,,,
1,313,313,313
2,453,453,453
3,178,178,178
4,44,44,44
5,11,11,11
6,1,1,1


To begin, we calculate the mean and standard deviation of the log-income for each word:

In [6]:
import numpy as np
import math

jobs = lending[['emp_title','annual_inc']][:74771]
jobs['words'] = jobs['emp_title'].str.split()
jobs = jobs[jobs['annual_inc'] > 0]

inc = []
words = []
for i in jobs.index:
   if type(jobs['words'][i]) is list:
      words = words + jobs['words'][i]
      inc = inc + len(jobs['words'][i]) * [math.log(jobs['annual_inc'][i])]
words_np = np.transpose(np.array([words, inc]))
words_df = pd.DataFrame(words_np, columns=['word','annual_inc'])
words_df['annual_inc'] = words_df['annual_inc'].apply(pd.to_numeric)
words_df.dtypes
words_df = words_df.groupby(['word']).agg(['mean', 'count', 'std']).sort_values(by=('annual_inc','count'), ascending=False)
with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
   print words_df
#words_df.groupby(['word'])['annual_inc'].mean()

                                        annual_inc                 
                                              mean  count       std
word                                                               
manager                                  11.244641  11967  0.438569
director                                 11.478994   2884  0.408701
assistant                                10.907341   2820  0.464425
sales                                    11.228467   2798  0.514542
supervisor                               11.075789   2251  0.400385
teacher                                  11.030307   2227  0.380377
driver                                   11.000896   2056  0.386418
specialist                               11.030591   2041  0.419578
senior                                   11.435916   1988  0.385810
owner                                    11.207871   1900  0.507399
engineer                                 11.455760   1884  0.328704
analyst                                  11.2518

We can use this table to find the average income for any job-word:

In [13]:
math.exp(words_df[('annual_inc','mean')]['teacher'])

61716.49446075176

Finally, I applied the model to the test data:

In [14]:
jobs_test = lending[['emp_title','annual_inc']][74771:]
jobs_test = jobs_test[pd.notnull(jobs_test['emp_title'])]
jobs_test['words'] = jobs_test['emp_title'].str.split()
jobs_test['annual_inc_pred'] = 0

In [15]:
for i in jobs_test.index:
   if jobs_test['words'][i] != np.nan:
      pred = []
      weight = []
      for word in jobs_test['words'][i]:
         if word in words_df[('annual_inc','mean')]:
            pred.append(words_df[('annual_inc','mean')][word])
            weight.append(math.sqrt(words_df[('annual_inc','count')][word])/words_df[('annual_inc','std')][word])
   if len(pred) > 0:
      jobs_test['annual_inc_pred'][i] = math.exp(np.average(np.array(pred), weights=weight))
   else:
      jobs_test['annual_inc_pred'][i] = jobs['annual_inc'].mean()
jobs_test['annual_inc_pred'][i]
jobs_test

/usr/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  


,emp_title,annual_inc,words,annual_inc_pred
83630,it manager,100000.0,"[it, manager]",79043.000000
83631,respiratory therapist,70000.0,"[respiratory, therapist]",70018.000000
83632,director of the children's program,41500.0,"[director, of, the, children's, program]",91803.000000
83634,it department,37000.0,"[it, department]",78210.000000
83635,front desk,25000.0,"[front, desk]",45597.000000
83636,checker,40000.0,[checker],48562.000000
83641,application developer,65000.0,"[application, developer]",88346.000000
83643,sales manager,108500.0,"[sales, manager]",76108.000000
83644,regional sales manager,88000.0,"[regional, sales, manager]",78444.000000
83645,"sr director, corporate member network",130000.0,"[sr, director,, corporate, member, network]",90165.000000


**Measuring model performance.** We see that our model does nearly as well as word2vec ensemble decision trees of the previous models.

In [17]:
print "MAE: " + str(abs(jobs_test['annual_inc_pred'] - jobs_test['annual_inc']).mean())

MAE: 23820.6220228


More importantly, our model here is <em>fundamentally different</em> than the word embedding models of the last section.  This model is expected to do well with common words, and do poorly for words that have not been seen frequently.  By contrast, the main advantage of the word2vec models was in generalizing from common words to less common words that occur in the same contexts.

Because the two models have such different strengths, we could likely combine them to create a superior composite model. This was my original goal in this section, however I will not have time. We will look at one type of ensembling in the next section.